# Traversable

## Definition and Laws

The `Traversable` type class in Haskell is one of the most powerful and elegant abstractions in functional programming. It generalizes the idea of iterating over a data structure, running an **effect** (an `Applicative` action) on each element, and then rebuilding the structure with the results.

While `Functor` allows you to modify values and `Foldable` allows you to squash values, `Traversable` allows you to **preserve the shape** of the data while processing the effects associated with the elements.

---
The definition can be found in `Data.Traversable`. To be a `Traversable`, a type must already be a `Functor` and a `Foldable`.

### The Formal Definition

```haskell
class (Functor t, Foldable t) => Traversable t where
    -- | Map each element of a structure to an action, 
    -- evaluate these actions from left to right, and collect the results.
    traverse :: Applicative f => (a -> f b) -> t a -> f (t b)
    traverse f = sequenceA . fmap f

    -- | Evaluate each action in the structure from left to right, 
    -- and collect the results.
    sequenceA :: Applicative f => t (f a) -> f (t a)
    sequenceA = ...

    -- Map-like versions (historical/specialized)
    mapM :: Monad m => (a -> m b) -> t a -> m (t b)
    mapM = traverse

    sequence :: Monad m => t (m a) -> m (t a)
    sequence = sequenceA

```

---

### Key Requirements & Methods

1. **`traverse`**: This is the most frequently used method. It takes a function that produces an effect (like `a -> Maybe b` or `a -> IO b`) and applies it to every element in the structure. It then "sequences" those effects together.
2. **`sequenceA`**: This is used when the effects are already inside the structure (e.g., `[Just 1, Just 2]`). It pulls the effect layer out to the front (`Just [1, 2]`).
3. **Minimal Complete Definition**: To create an instance of `Traversable`, you must implement **either** `traverse` or `sequenceA`.

### The Laws

For a `Traversable` instance to be valid, it must satisfy three laws for `traverse` that ensure the structure is preserved and the effects are handled predictably:

1. **Identity:** `traverse Identity = Identity`
    * Mapping the `Identity` constructor over a structure should change nothing.


2. **Composition:** `traverse (Compose . fmap g . f) = Compose . fmap (traverse g) . traverse f`
    * Traversing two functions one after the other is the same as traversing their composition.


3. **Naturality:** `t . traverse f = traverse (t . f)`
    * This ensures that changing the effect type (via a natural transformation `t`) doesn't change the way the structure is traversed.

Alternatively the `sequenceA` function must satisfy the following laws:

1. **Identity:** `sequenceA . fmap Identity = Identity`
2. **Composition:**  `sequenceA . fmap Compose = Compose . fmap sequenceA . sequenceA`
3. **Naturality:** `t . sequenceA = sequenceA . fmap t`

---

### Implementation Requirement

If you are defining your own instance of `Traversable`, your implementation of `sequenceA` must satisfy these rules. If it doesn't, other Haskell libraries (which assume these laws are true) might produce bugs when using your data type.

**Would you like to see a code example of how the Composition law looks with actual types like `Maybe` and `[]`?**


### 1. The `sequenceA` Function: Turning Structures Inside-Out

To understand `Traversable`, it is often easiest to start with `sequenceA`. Its primary job is to flip two contexts (layers of types). It turns a "structure of actions" into an "action that returns a structure."

#### The Type Signature

Let's break this down:

* **Input:** `t (f a)` — A Traversable structure `t` (like a List) containing Applicative effects `f` (like `Maybe` or `IO`).
* **Output:** `f (t a)` — The Applicative effect `f` is now on the outside, wrapping the structure `t`.

#### Intuition: The "Inside-Out" Flip

Imagine you have a **List** of **Maybe** integers: `[Just 1, Just 2, Just 3]`.

* The type is `[Maybe Int]` (List of Maybes).
* If you want to ensure *all* elements exist before processing the list, you want to turn this into `Maybe [Int]` (Maybe a List).

If any element is `Nothing`, the whole result becomes `Nothing`. If all are `Just`, you get `Just [1, 2, 3]`.

**Example:**

```haskell
ghci> sequenceA [Just 1, Just 2, Just 3]
Just [1,2,3]

ghci> sequenceA [Just 1, Nothing, Just 3]
Nothing

```

Here, `sequenceA` collected the effects (the `Maybe` context) and pulled them to the outer level.

---

### 2. Defining `traverse` using `sequenceA`

The `traverse` function is the heart of the `Traversable` type class. It combines mapping and sequencing into a single step.

However, per your request, we can define `traverse` by composing `fmap` and `sequenceA`.

#### The Definition

Or in Haskell code:

```haskell
traverse :: (Traversable t, Applicative f) => (a -> f b) -> t a -> f (t b)
traverse f xs = sequenceA (fmap f xs)

```

#### How it works (Step-by-Step)

Let's trace the types to see how this composition achieves the goal of `traverse`.

1. **The Goal:** We start with `t a` and a function `(a -> f b)`. We want to end up with `f (t b)`.
2. **Step 1 (`fmap f`):**
* We apply the function `f` to every element inside the structure `t`.
* This transforms `t a` into `t (f b)`.
* Now we have a structure where every element is wrapped in an effect.


3. **Step 2 (`sequenceA`):**
* We take the result from step 1, which is `t (f b)`.
* We apply `sequenceA`, which flips the layers.
* This transforms `t (f b)` into `f (t b)`.



#### Visualizing the Transformation

Imagine a function `check :: Int -> Maybe Int` that returns `Nothing` if a number is odd, and `Just n` if it is even. We want to traverse a list `[2, 4]`.

1. **Start:** `[2, 4]`
2. **`fmap check`:** `[Just 2, Just 4]` (We now have a List of Maybes)
3. **`sequenceA`:** `Just [2, 4]` (We now have a Maybe List)

---

## Examples

To truly understand `Traversable`, it helps to see it applied to a custom data structure. Let’s move away from lists and look at a **Binary Tree**.

A Tree is `Traversable` because it has a fixed shape (nodes and leaves), and we can visit every element in a specific order (e.g., left-to-right).

---

### 1. The Data Type

First, we define a simple Tree and its `Functor` and `Foldable` instances (which are prerequisites).

```haskell
data Tree a = Leaf | Node (Tree a) a (Tree a)
  deriving (Show, Eq)

instance Functor Tree where
    fmap _ Leaf = Leaf
    fmap f (Node l x r) = Node (fmap f l) (f x) (fmap f r)

instance Foldable Tree where
    foldMap _ Leaf = mempty
    foldMap f (Node l x r) = foldMap f l <> f x <> foldMap f r

```

---

### 2. Implementing the Traversable Instance

To make `Tree` traversable, we implement `traverse`. Notice how the implementation looks almost exactly like `fmap`, but we use the Applicative **`ficiton` operator (`<*>`)** and **`pure`** to rebuild the tree.

```haskell
instance Traversable Tree where
    -- traverse :: Applicative f => (a -> f b) -> Tree a -> f (Tree b)
    traverse _ Leaf = pure Leaf
    traverse f (Node l x r) = 
        Node <$> traverse f l <*> f x <*> traverse f r

```

---

### 3. Concrete Example: Validation

Imagine you have a tree of integers, and you want to ensure **all** numbers are even. If any number is odd, the whole operation should fail (return `Nothing`).

#### The Setup

```haskell
-- A function that returns an effect (Maybe)
ensureEven :: Int -> Maybe Int
ensureEven n 
    | even n    = Just n
    | otherwise = Nothing

-- Two example trees
validTree   = Node (Node Leaf 2 Leaf) 4 (Node Leaf 6 Leaf)
invalidTree = Node (Node Leaf 2 Leaf) 3 (Node Leaf 6 Leaf)

```

#### The Result

When we `traverse` the tree with `ensureEven`:

```haskell
ghci> traverse ensureEven validTree
Just (Node (Node Leaf 2 Leaf) 4 (Node Leaf 6 Leaf))

ghci> traverse ensureEven invalidTree
Nothing

```

**What happened here?**

* `traverse` walked the tree.
* It applied `ensureEven` to every node.
* It collected all the `Just` values and rebuilt the **exact same tree shape** inside a single `Just`.
* Because `invalidTree` had a `3`, that specific node returned `Nothing`, which caused the entire result to become `Nothing`.

---

### 4. Concrete Example: Using `sequenceA`

If you already have a tree where the nodes are already wrapped in effects (e.g., `Tree (IO Int)`), you use `sequenceA` to "run" the tree.

```haskell
-- A tree where every node is an IO action
actionTree :: Tree (IO String)
actionTree = Node Leaf (getLine) Leaf

-- Pull the IO out: This creates ONE IO action that, 
-- when run, produces a Tree of Strings.
combinedAction :: IO (Tree String)
combinedAction = sequenceA actionTree

```

---

### Why this is better than a List

With a List, it's easy to think `traverse` is just "mapping and flattening." But with a **Tree**, you see that the **structure is sacred**. `Traversable` guarantees that if you start with a Tree, you end with a Tree—never a List or a single value—while allowing the effects (like `Maybe`, `IO`, or `Either`) to determine if the final structure is even created.

## Chapter Exercises

### Traversable instances
Write a Traversable instance for the datatype provided, filling in any required superclasses. Use `QuickCheck` to validate your instances.

#### Identity
Write a Traversable instance for `Identity`.

In [22]:
newtype Identity a = Identity a deriving (Eq, Ord, Show)

instance Functor Identity where
    fmap f (Identity x) = Identity (f x)

instance Foldable Identity where
    foldMap f (Identity x) = f x

instance Traversable Identity where
    sequenceA (Identity x) = Identity <$> x